In [32]:
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import nltk
from IPython.display import display
pd.options.display.max_columns = None
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [2]:
# Import the datasets to work with.
df = pd.read_csv('wiki_movie_plots_deduped.csv')
df_genre = pd.read_csv("title.basics.tsv",delimiter="\t")

/usr/local/Cellar/ipython/7.0.1/libexec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Lets review the shapes of the dataframes.
display(df.shape)
display(df_genre.shape) # This one is much longer 

(34886, 8)

(5463542, 9)

In [4]:
# Lets review a small sample of the dataframe to see what we have. 
display(df.head(2))
display(df_genre.head(2))

Release Year                          Title Origin/Ethnicity Director Cast  \
0          1901         Kansas Saloon Smashers         American  Unknown  NaN   
1          1901  Love by the Light of the Moon         American  Unknown  NaN   

     Genre                                          Wiki Page  \
0  unknown  https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...   
1  unknown  https://en.wikipedia.org/wiki/Love_by_the_Ligh...   

                                                Plot  
0  A bartender is working at a saloon, serving dr...  
1  The moon, painted with a smiling face hangs ov...

tconst titleType            primaryTitle           originalTitle  \
0  tt0000001     short              Carmencita              Carmencita   
1  tt0000002     short  Le clown et ses chiens  Le clown et ses chiens   

   isAdult startYear endYear runtimeMinutes             genres  
0        0      1894      \N              1  Documentary,Short  
1        0      1892      \N              5    Animation,Short

In [5]:
# From the df_genre dataframe i just need the title of the movie and the genres column, so lets just work with that.
df_genre_used = df_genre[["primaryTitle","genres"]]
df_genre_used = df_genre_used.rename(columns={"primaryTitle":"Title"}) # I

In [6]:
# Lets merge the dfs to add the genres to our original df
df_joined = df.merge(df_genre_used,on="Title").iloc[:20000]

In [7]:
# The original df had 34.886 rows only, the merged one has 261.399 rows, this mean that there are repeated movies.
display("shape of joined df: {}".format(df_joined.shape))

# Lets see how many unique movies we had (should be  <= 34.886 because the joined do a inner join)
display("Unique movies: {}".format(len(df_joined["Title"].unique().tolist())))

# Example of a repeated movie (42 times): 
jack = len(df_genre_used[df_genre_used["Title"]=="Jack and the Beanstalk"])
display("For example,the movie Jack and the Beanstalk appears {} times".format(jack))

'shape of joined df: (20000, 9)'

'Unique movies: 1033'

'For example,the movie Jack and the Beanstalk appears 42 times'

In [8]:
# Indeed, there are movies that repeat many times that we must clean. Lets just keep the columns that i need.
df_joined_c = df_joined.loc[:,("Release Year","Title","Plot","genres")]
# Lets drop duplicate rows, There were almost 95 thousand repeated rows (261399 - 167621).
df_joined_c.drop_duplicates(inplace=True)
display(df_joined_c.shape)
df_joined_c.head(3)

(11216, 4)

Release Year                          Title  \
0          1901         Kansas Saloon Smashers   
1          1901  Love by the Light of the Moon   
2          1901        The Martyred Presidents   

                                                Plot                  genres  
0  A bartender is working at a saloon, serving dr...            Comedy,Short  
1  The moon, painted with a smiling face hangs ov...  Comedy,Fantasy,Romance  
2  The film, just over a minute long, is composed...                   Short

In [9]:
# Lets create a function to select the most repeated genres for a Title.
def get_best_genres(movie):
    """The function transform an array of genres to a tuple of a maximum of the 3 most repeated
    genres for a given Title"""    
    genres = df_joined_c[df_joined_c["Title"]== movie]["genres"].tolist() # List of genres
    list_genres = (",".join(genres)).split(",")
    return pd.Series(list_genres).value_counts().index[:3].tolist()

get_best_genres("The Martyred Presidents")   

['Short']

In [10]:
#df_processed = (pd.DataFrame([{"Release_Year":np.nan,"Title":np.nan,"Plot":np.nan,"Genres":np.nan}])).dropna()
empty = []
def genres(movie):    
    genres = df_joined_c[df_joined_c["Title"]== movie]["genres"].tolist() # List of genres
    rel_year = df_joined_c[df_joined_c["Title"]== movie]["Release Year"].iloc[0] # real year 
    title = df_joined_c[df_joined_c["Title"]== movie]["Title"].iloc[0] # title
    plot = df_joined_c[df_joined_c["Title"]== movie]["Plot"].iloc[0] # plot
    list_genres = (",".join(genres)).split(",") # genres in list 
    genres_proc = pd.Series(list_genres).value_counts().index[:3].tolist() # Top 3 most repeated genres
    inf = {"Release_Year":rel_year,"Title":title,"Plot":plot,"Genres":genres_proc} # Final row to add
    return empty.append(inf)


In [11]:
%%time
unique_titles = df_joined_c["Title"].unique().tolist()
for title in unique_titles:
    genres(title)

CPU times: user 12.6 s, sys: 99.5 ms, total: 12.7 s
Wall time: 16.1 s


In [37]:
movies = pd.DataFrame(empty)
movies.head()

Genres  \
0              [Comedy, Short]   
1   [Comedy, Romance, Fantasy]   
2                      [Short]   
3              [Comedy, Short]   
4  [Family, Animation, Comedy]   

                                                Plot  Release_Year  \
0  A bartender is working at a saloon, serving dr...          1901   
1  The moon, painted with a smiling face hangs ov...          1901   
2  The film, just over a minute long, is composed...          1901   
3  Lasting just 61 seconds and consisting of two ...          1901   
4  The earliest known adaptation of the classic f...          1902   

                              Title  
0            Kansas Saloon Smashers  
1     Love by the Light of the Moon  
2           The Martyred Presidents  
3  Terrible Teddy, the Grizzly King  
4            Jack and the Beanstalk

In [38]:
#movies["Genres"].str.split(',',expand=True)
movies["Genres_processed"]= movies["Genres"].apply(lambda x: ",".join(map(str,x)))

In [39]:
movies[["gen1","gen2","gen3"]] = movies["Genres_processed"].str.split(",",expand=True)
a = list(set(movies["gen1"].tolist()))
b = list(set(movies["gen2"].tolist()))
c = list(set(movies["gen3"].tolist()))
display("Amount of genres: {}".format(len(list(set(a+b+c)))))
genre_list= list(set(a+b+c))

'Amount of genres: 29'

In [40]:
for genre in genre_list:
    movies[genre] = movies["Genres"].apply(lambda x: 1 if genre in x else 0)

In [41]:
movies_cleaned = movies.drop(["Genres","Genres_processed","gen1","gen2","gen3","\\N",np.nan],axis=1)

In [42]:
movies_cleaned["Release_Year"] = movies_cleaned["Release_Year"].astype("category")

In [43]:
def tokenize(s):
    return word_tokenize(s)

def stem_and_lemmatize(l):
    ps = PorterStemmer()
    stemmed = [ps.stem(word).lower() for word in l]
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in stemmed]

def remove_stopwords(l):
    stopWords = set(stopwords.words('english'))
    morewords = ["``","''","'s",] 
    filtered = [word for word in l if word not in stopWords and word not in morewords and len(word)>1]
    return filtered

In [44]:
movies_cleaned["Plot"] = movies_cleaned["Plot"].apply(tokenize)

In [45]:
movies_cleaned["Plot"] = movies_cleaned["Plot"].apply(stem_and_lemmatize)

In [46]:
movies_cleaned["Plot"] = movies_cleaned["Plot"].apply(remove_stopwords)

In [47]:
words = movies_cleaned["Plot"].tolist()
all_words = [word2 for word in words for word2 in word]
all_words2 = nltk.FreqDist(all_words)
most_common_words = [w[0] for w in all_words2.most_common(1000)]

In [50]:
for word in most_common_words:
    movies_cleaned[word] = movies_cleaned["Plot"].apply(lambda x: 1 if word in x else 0)

In [54]:
movies_cleaned.head(2)

Plot Release_Year  \
0  [bartend, work, saloon, serv, drink, custom, f...         1901   
1  [moon, paint, smile, face, hang, park, night, ...         1901   

                           Title  Fantasy  Drama  Animation  Crime  Western  \
0         Kansas Saloon Smashers        0      0          0      0        0   
1  Love by the Light of the Moon        1      0          0      0        0   

   Romance  Short  Comedy  Musical  Documentary  Music  Action  Biography  \
0        0      1       1        0            0      0       0          0   
1        1      0       1        0            0      0       0          0   

   Talk-Show  Reality-TV  Film-Noir  Sport  News  Adventure  Family  Thriller  \
0          0           0          0      0     0          0       0         0   
1          0           0          0      0     0          0       0         0   

   Adult  History  Horror  War  Mystery  Sci-Fi  hi  ha  love  marri  take  \
0      0        0       0    0        0       0   1   0     0      0     0   
1      0        0       0    0        0       0   1   1     0      0     0   

   find  get  one  man  leav  return  father  becom  tell  film  back  howev  \
0     0    0    0    1     1       0       0      0     0     0     0      0   
1     0    1    0    1     0       0       0      0     0     0     0      0   

   woman  two  young  see  tri  girl  fall  home  mari  go  make  wa  wife  \
0      0    0      0    0    0     0     0     0     0   0     0   0     0   
1      1    0      1    1    0     0     0     0     0   0     0   0     0   

   come  play  new  time  day  goe  kill  friend  later  thi  away  give  \
0     0     0    0     0    0    0     0       0      0    0     0     0   
1     0     0    0     0    0    0     0       0      0    0     0     0   

   arriv  meet  daughter  befor  john  onli  decid  doe  work  end  show  \
0      0     0         0      1     0     0      0    0     1    0     0   
1      0     0         0      0     0     0      0    0     0    0     0   

   live  hous  mother  husband  life  child  discov  son  famili  money  help  \
0     0     0       0        0     0      0       0    0       0      0     0   
1     0     0       0        0     0      0       0    0       0      0     0   

   escap  final  attempt  dure  anoth  also  ask  night  first  men  brother  \
0      0      0        0     0      0     0    0      0      0    0        0   
1      0      0        0     0      0     0    0      1      0    0        0   

   jim  follow  order  polic  save  turn  run  plan  marriag  want  eventu  \
0    0       1      1      0     0     0    0     0        0     0       0   
1    0       0      0      0     0     0    0     0        0     0       0   

   way  die  use  manag  room  realiz  next  name  year  meanwhil  set  boy  \
0    0    0    0      0     0       0     0     0     0         0    0    0   
1    0    0    0      0     0       0     0     0     0         0    0    0   

   soon  forc  learn  death  fight  murder  joe  town  wealthi  mrs.  place  \
0     0     0      0      0      0       0    0     0        0     0      0   
1     0     0      1      0      0       0    0     0        0     0      0   

   rescu  believ  offic  break  agre  parti  refus  offer  reveal  ship  \
0      0       0      0      1     0      0      0      0       0     0   
1      0       0      0      0     0      0      0      0       0     0   

   train  war  american  begin  becaus  call  win  visit  citi  stori  sister  \
0      0    0         0      1       0     0    0      0     0      0       0   
1      0    0         0      0       1     0    0      0     0      0       0   

   jack  old  know  say  send  tom  captain  even  coupl  fire  head  prison  \
0     0    0     0    0     0    0        0     0      0     0     1       0   
1     0    0     0    0     0    0        0     0      1     0     0       0   

   littl  happi  describ  princ  onc  start  harol

In [59]:
X = movies_cleaned.iloc[:,30:]
Y = movies_cleaned.iloc[:,3:30]



Fantasy  Drama  Animation  Crime  Western  Romance  Short  Comedy  Musical  \
0        0      0          0      0        0        0      1       1        0   
1        1      0          0      0        0        1      0       1        0   
2        0      0          0      0        0        0      1       0        0   
3        0      0          0      0        0        0      1       1        0   
4        0      0          1      0        0        0      0       1        0   

   Documentary  Music  Action  Biography  Talk-Show  Reality-TV  Film-Noir  \
0            0      0       0          0          0           0          0   
1            0      0       0          0          0           0          0   
2            0      0       0          0          0           0          0   
3            0      0       0          0          0           0          0   
4            0      0       0          0          0           0          0   

   Sport  News  Adventure  Family  Thriller  Adult  History  Horror  War  \
0      0     0          0       0         0      0        0       0    0   
1      0     0          0       0         0      0        0       0    0   
2      0     0          0       0         0      0        0       0    0   
3      0     0          0       0         0      0        0       0    0   
4      0     0          0       1         0      0        0       0    0   

   Mystery  Sci-Fi  
0        0       0  
1        0       0  
2        0       0  
3        0       0  
4        0       0